In [1]:
from tools import *
import json
#from webapp import *
import pyodbc
#IMPORT DATABASE FROM SQL SERVER AND FIT DATA JUST OFF OF THE DATABASE

In [2]:
server = 'wdtdevsql02'
database = 'WDT_PROD'
username= 'application'
password= '!v80VD8gX!vK'
pyodbc_cnntr = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password

In [3]:
'''server: wdtdevsql02
database: WDT_PROD
login: application
password: !v80VD8gX!vK'''

'server: wdtdevsql02\ndatabase: WDT_PROD\nlogin: application\npassword: !v80VD8gX!vK'

In [20]:
to_return = {}  
cnxn = pyodbc.connect(pyodbc_cnntr)
cursor = cnxn.cursor()    
cursor.execute("SELECT Frequency_Hz, Minus_Z_Imaginary_Ohm, Z_Real_Ohm FROM ACImpedanceData_Raw WHERE Cycle = 1 AND ProcessDataID = 737864")
for row in cursor.fetchall():
    to_return.update({row.Frequency_Hz: [row.Z_Real_Ohm, row.Minus_Z_Imaginary_Ohm]})
cnxn.close()

In [21]:
import pprint
pp = pprint.PrettyPrinter(width=41, compact=True)
to_return

{1.0000637: [25524996.0, 5322199.0],
 1.1224855: [25198394.0, 5468659.0],
 1.2596743: [24708566.0, 5800842.0],
 1.4122375: [24416218.0, 5929125.5],
 1.5840428: [24262230.0, 5756475.5],
 1.7796121: [23679500.0, 6118924.5],
 1.9970596: [23172492.0, 6331669.5],
 2.2398219: [22821876.0, 6360373.5],
 2.513674: [22431738.0, 6654888.5],
 2.8183618: [22018528.0, 6734998.5],
 3.1603959: [21492932.0, 6943168.5],
 3.5511355: [20987878.0, 6915280.5],
 3.9778507: [20426124.0, 7380130.0],
 4.4693928: [19743972.0, 7714598.0],
 5.0144405: [19489748.0, 8027859.5],
 5.6205029: [19501802.0, 7654888.5],
 6.3105812: [17957812.0, 7786491.5],
 7.0765405: [17295566.0, 8261442.0],
 7.9395342: [16700054.0, 8772757.0],
 8.9183798: [16152548.0, 8554055.0],
 10.003201: [15760374.0, 8834763.0],
 11.224855: [14564249.0, 8894460.0],
 12.600808: [13724772.0, 9258300.0],
 14.118973: [12983082.0, 9263558.0],
 15.857575: [11527969.0, 9403910.0],
 17.796127: [11302319.0, 9344952.0],
 19.955299: [10578121.0, 9263836.0],
 2

In [54]:
freq = [np.float64(i) for i in list(to_return.keys())]
re = [np.float64(i[0]) for i in list(to_return.values())]
im = [np.float(i[1]) for i in list(to_return.values())]

In [56]:
#self, params, circuit, weight_func='modulus', nan_policy='raise', maxfev = 10
#minimize(self.leastsq_errorfunc, params, method='leastsq', args=(self.df[i].w.values, self.df[i].re.values, self.df[i].im.values, circuit, weight_func), nan_policy=nan_policy, maxfev=maxfev)
type(freq[0])

numpy.float64

In [47]:
Rs_guess = min(re)
R1_guess = max(re)//4
n1_guess = random.uniform(0, 1)
q1_guess = random.uniform(0, .001)
R2_guess = 2*max(re)//4
n2_guess = random.uniform(0, 1)
q2_guess = random.uniform(0, .001)
Q3_guess = random.uniform(0, .001)
n3_guess = random.uniform(0, 1)
params = Parameters()
params.add('Rs', value=Rs_guess, min=Rs_guess*.001, max=Rs_guess*10)
params.add('R1', value=R1_guess, min=R1_guess*.001, max=R1_guess*10)
params.add('n1', value=n1_guess, min=0, max=1)
params.add('Q1', value=q1_guess, min=0, max=.001)
params.add('R2', value=R2_guess, min=R2_guess*.001, max=R2_guess*10)
params.add('n2', value=n2_guess, min=.01, max=1)
params.add('Q2', value=q2_guess, min=0, max=.001)
params.add('Q', value=Q3_guess, min=0, max=.001)
params.add('n', value=n3_guess, min=.01, max=1)

In [48]:
def leastsq_errorfunc(params, w, re, im, circuit, weight_func = 'modulus'):
        if circuit == 'R-RQ-RQ':
            re_fit = cir_RsRQRQ_fit(params, w).real
            im_fit = -cir_RsRQRQ_fit(params, w).imag
        elif circuit == 'R-RQ-Q':
            re_fit = cir_RsRQQ_fit(params, w).real
            im_fit = -cir_RsRQQ_fit(params, w).imag
        elif circuit == 'R-RQ-RQ2':
            re_fit = cir_RsRQRQ2_fit(params, w).real
            im_fit = -cir_RsRQRQ2_fit(params, w).imag
        elif circuit == 'R-RQ-RQ-Q':
            re_fit = cir_RsRQRQQ_fit(params, w).real
            im_fit = -cir_RsRQRQQ_fit(params, w).imag
        else:
            print('Circuit is not defined in leastsq_errorfunc()')

        error = ([(re-re_fit)**2, (im-im_fit)**2]) #sum of squares
        
        #Different Weighing options, see Lasia
        if weight_func == 'modulus':
            weight = [1/((re_fit**2 + im_fit**2)**(1/2)), 1/((re_fit**2 + im_fit**2)**(1/2))]
        elif weight_func == 'proportional':
            weight = [1/(re_fit**2), 1/(im_fit**2)]
        elif weight_func == 'unity':
            unity_1s = []
            for k in range(len(re)):
                unity_1s.append(1) #makes an array of [1]'s, so that the weighing is == 1 * sum of squres.
            weight = [unity_1s, unity_1s]
        else:
            print('weight not defined in leastsq_errorfunc()')
        
        S = np.array(weight) * error #weighted sum of squares 
        return S
    

In [59]:
circuit = 'R-RQ-RQ-Q'
weight_func = 'modulus'
nan_policy = 'raise'
maxfev = 10
minimize(leastsq_errorfunc, params, method='leastsq', args=(freq, re, im, circuit, weight_func), nan_policy=nan_policy, maxfev=maxfev)

TypeError: can't multiply sequence by non-int of type 'complex'

In [82]:
pd.Series(freq)*1j

0      0.000000e+00+1.000064e+00j
1      0.000000e+00+1.122486e+00j
2      0.000000e+00+1.259674e+00j
3      0.000000e+00+1.412238e+00j
4      0.000000e+00+1.584043e+00j
5      0.000000e+00+1.779612e+00j
6      0.000000e+00+1.997060e+00j
7      0.000000e+00+2.239822e+00j
8      0.000000e+00+2.513674e+00j
9      0.000000e+00+2.818362e+00j
10     0.000000e+00+3.160396e+00j
11     0.000000e+00+3.551136e+00j
12     0.000000e+00+3.977851e+00j
13     0.000000e+00+4.469393e+00j
14     0.000000e+00+5.014441e+00j
15     0.000000e+00+5.620503e+00j
16     0.000000e+00+6.310581e+00j
17     0.000000e+00+7.076541e+00j
18     0.000000e+00+7.939534e+00j
19     0.000000e+00+8.918380e+00j
20     0.000000e+00+1.000320e+01j
21     0.000000e+00+1.122485e+01j
22     0.000000e+00+1.260081e+01j
23     0.000000e+00+1.411897e+01j
24     0.000000e+00+1.585758e+01j
25     0.000000e+00+1.779613e+01j
26     0.000000e+00+1.995530e+01j
27     0.000000e+00+2.238539e+01j
28     0.000000e+00+2.510443e+01j
29     0.00000

In [80]:
masked_mpt = mpt_data(['copy.mpt'])
type(masked_mpt.df[0].w)

pandas.core.series.Series

In [58]:
type(freq[0])

numpy.float64